In [1]:
!nvidia-smi

Mon Sep 11 16:18:57 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   32C    P0              69W / 300W |   6248MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# !ls -lah /root/.cache/huggingface/hub/

In [3]:
!ls -lah /root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese

total 2.5G
drwxr-xr-x 2 root root  237 Aug 31 10:56 .
drwxr-xr-x 4 root root   76 Aug 31 10:56 ..
-rw-r--r-- 1 root root 1.5K Aug 31 09:35 .gitattributes
-rw-r--r-- 1 root root  443 Aug 31 09:35 README.md
-rw-r--r-- 1 root root  821 Aug 31 09:35 config.json
-rw-r--r-- 1 root root   69 Aug 31 09:35 eval_results.txt
-rw-r--r-- 1 root root 1.3G Aug 31 10:15 model.safetensors
-rw-r--r-- 1 root root 1.3G Aug 31 10:56 pytorch_model.bin
-rw-r--r-- 1 root root  125 Aug 31 10:56 special_tokens_map.json
-rw-r--r-- 1 root root 430K Aug 31 10:56 tokenizer.json
-rw-r--r-- 1 root root  514 Aug 31 10:56 tokenizer_config.json
-rw-r--r-- 1 root root 107K Aug 31 10:56 vocab.txt


In [4]:
# import sys,os,os.path
# os.environ['HTTP_PROXY']="http://127.0.0.1:8098"
# os.environ['HTTPS_PROXY']="http://127.0.0.1:8098"

In [5]:
import torch
torch.cuda.device_count()

1

In [6]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [7]:
# https://huggingface.co/bigscience/bloom-1b7

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [9]:
# from transformers import BloomTokenizerFast

In [10]:
# tokenizer = BloomTokenizerFast.from_pretrained(
#     "bigscience/bloom-1b7",
#     local_files_only=True
# )

In [11]:
model_name = "FlagAlpha/Llama2-Chinese-7b-Chat"

In [12]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name,
                                          local_files_only=True
                                          # use_auth_token=True,
                                         )

In [13]:
tokenizer.eos_token_id

2

In [14]:
tokenizer.pad_token_id

In [15]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             local_files_only=True,
                                             device_map='auto',
                                             # torch_dtype=torch.float16,
                                             temperature=0.2, # must be strictly positive float
                                             do_sample=True,
                                             # use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )

2023-09-11 16:19:00.912072: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 16:19:01.606064: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [16]:
!nvidia-smi

Mon Sep 11 16:19:16 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   32C    P0              71W / 300W |  32459MiB / 46068MiB |      5%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
# https://github.com/shibing624/text2vec/issues/116

In [18]:
from torch.hub import _get_torch_home

_get_torch_home()

'/root/.cache/torch'

In [19]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="/root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese",
                                                      # local_files_only=True,
                                                      model_kwargs={"device": "cuda"})

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.


In [20]:
instructor_embeddings

HuggingFaceInstructEmbeddings(client=INSTRUCTOR(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
), model_name='/root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese', cache_folder=None, model_kwargs={'device': 'cuda'}, encode_kwargs={}, embed_instruction='Represent the document for retrieval: ', query_instruction='Represent the question for retrieving supporting documents: ')

In [21]:
!nvidia-smi

Mon Sep 11 16:19:20 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   32C    P0              69W / 300W |  32459MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [23]:
from langchain.vectorstores import Chroma
persist_directory = 'db_cn'

vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=instructor_embeddings)

retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [24]:
q2 = "在哪里可以设定机种?"

In [25]:
retriever_k = vectordb.as_retriever(
    search_kwargs={'k': 3}
)

In [26]:
ans2 = retriever_k.get_relevant_documents(q2)

In [27]:
len(ans2)

3

In [28]:
ans2

[Document(page_content='选择是否设定附加规则 , 限制 SN长度或厂家代码范围等 , 设定完毕后 , 点击【保存】 , \n完成该产品序列号判定逻辑设定 .', metadata={'source': 'chinese_pdf/保修管理-廠家sop-repair(去圖標).pdf', 'page': 7}),
 Document(page_content='选定不良代码 , 点击进入客户绑定页面 ; 勾选客户 , 点击【保存】完成不良判定代码\n与客户的绑定 ; \n \n \n保修设定  \n1. 定义产品 (整机料号信息 ) \n单笔或批量将产品信息导入系统 ,包含机种、厂家料号、客户料号、供应商料号、替代\n料号等信息  \n1.1 机种管理  \n若产品涉及机种 , 需先将机种信息新增至系统 , 以便后续选择 . \na. 单笔新增机种  \n厂家保修管理 后台 ->保修设定 ->定义产品 , 点击【机种管理】按钮 , 点击【新增】填写\n机种名称 , 点击【保存】完成 ;', metadata={'source': 'chinese_pdf/保修管理-廠家sop-repair(去圖標).pdf', 'page': 10}),
 Document(page_content='1.9 产品与附件绑定  \n本操作步骤 , 需要在本章 2. 定义附件  操作完成后进行 . \n厂家保修管理 后台 ->保修设定 ->定义产品 , 选定产品 , 点击按钮 , 点击【绑定附件】并勾\n选对应附 件、填写数量 , 点击【绑定】完成 ; \n \n \n2. 定义部件 （备品料号信息）  \n若部件中涉及机种管理 , 亦需先新增或上传机种 , 而后操作步骤同定义产品  \n2.1 新增部件机种  \n点击【机种管理】按钮 , 点击【新增】单笔新增机种 ; 点击【上传机种】下载模板 , 批\n量新增机种 ;  \n2.2 新增部件  \n点击【新增部件】单笔新增部件 ; 点击【上传部件】下拉框 , 下载模板批量新增部件 ; \n2.3 新增部件客户料号 ; \n点击【料号对照】添加客户料号 , 选定产品 , 点击单笔添加 ; 点击【料号对照】下载模\n板, 批量上传料号 ; \n2.3 新增供应商料号  \n点击【供应商料号】 , 选定产品 , 点击

In [29]:
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [30]:
# https://python.langchain.com/docs/modules/model_io/models/chat/prompts

In [31]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                # return_full_text=True,
                # torch_dtype=torch.bfloat16,
                # device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [32]:
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

In [33]:
llm("你好嗎?")

/usr/local/lib/python3.10/site-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


'\nЋ：你好，我是来自中国的学生。\nA：很高兴认识你，我是来自美国的学生。你在这里学习了哪些科目？\nЋ：我学习了数学、物理和化学。你在这里学习了什么科目？\nA：我学习了历史、政治和经济学。你认为这些科目对你的未来有哪些帮助？\nЋ：我认为这些科目对我的未来有很大的帮助，因为它们可以帮助我更好地理解世界和人类的行为。\nA：我也认为这些科目对我的未来有很大的帮助，因为它们可以帮助我更好地理解世界和人类的行为。\nЋ：我们有很多共同的兴趣，我们可以一起参加一些学术活动。\nA：我也非常感谢你的提议，我们可以一起参加一些学术活动。\n\n\n基于以上这段对话内容回答：\n这两个学生之间有什么共同点吗？\n'

In [34]:
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain

In [64]:
from langchain.prompts import PromptTemplate

In [77]:
template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。

  {context}

  问题: {queston}
  答案:"""


prompt_template = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

chain_type_kwargs = {
    "prompt": prompt_template
}

In [81]:
template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。

  {context}

  问题: {question}
  Answer in English:"""


prompt_template = PromptTemplate.from_template(
    template=template
)

chain_type_kwargs = {
    "prompt": prompt_template
}

In [82]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True,
                                 chain_type_kwargs=chain_type_kwargs
                                )

In [83]:
import json
import pprint

# 1

In [84]:
question1 = "如何设定判定逻辑?"

In [85]:
%%time
res1 = qa({"query": question1})

CPU times: user 12.5 s, sys: 3.12 s, total: 15.6 s
Wall time: 15.6 s


In [86]:
res1

{'query': '如何设定判定逻辑?',
 'result': ' \n\nTo set the judgment logic, you can follow these steps:\n\n1. Go to the "Basic Settings" page in the "Firmware Management" system.\n2. Click on the "Add/Edit Judgment Logic" button.\n3. Select the "PPID/SN" judgment rule.\n4. Fill in the judgment rule name, description, and SN sample.\n5. Click on the "Edit" button to apply the changes.\n6. Fill in the decode rule name, description, and SN sample.\n7. Click on the "Add" button to add the judgment rule.\n8. Fill in the PPID/SN sub-string section, based on the meaning of the sub-string.\n9. Click on the "Edit" button to edit the judgment rule.\n10. Fill in the monthly rule for the PPID/SN sub-string section.\n11. Click on the "Add a new item" button to add a new rule.\n12. Fill in the PPID/SN sub-string rule.\n\nNote: The above steps are based on the Chinese version of the instruction manual. Please refer to the English version for the exact steps.',
 'source_documents': [Document(page_content='选择是否

In [87]:
res1["result"]

' \n\nTo set the judgment logic, you can follow these steps:\n\n1. Go to the "Basic Settings" page in the "Firmware Management" system.\n2. Click on the "Add/Edit Judgment Logic" button.\n3. Select the "PPID/SN" judgment rule.\n4. Fill in the judgment rule name, description, and SN sample.\n5. Click on the "Edit" button to apply the changes.\n6. Fill in the decode rule name, description, and SN sample.\n7. Click on the "Add" button to add the judgment rule.\n8. Fill in the PPID/SN sub-string section, based on the meaning of the sub-string.\n9. Click on the "Edit" button to edit the judgment rule.\n10. Fill in the monthly rule for the PPID/SN sub-string section.\n11. Click on the "Add a new item" button to add a new rule.\n12. Fill in the PPID/SN sub-string rule.\n\nNote: The above steps are based on the Chinese version of the instruction manual. Please refer to the English version for the exact steps.'

# 2

In [45]:
question2 = "在哪里可以设定机种?"

In [46]:
%%time
res2 = qa({"query": question2})

CPU times: user 2.87 s, sys: 975 ms, total: 3.84 s
Wall time: 3.85 s


In [47]:
res2["result"]

' 在厂家保修管理 后台 -> 保修设定 -> 定义产品 操作中，可以在 1.1 机种管理 操作中设定机种。\n'

In [56]:
question3 = "平台可以自动核准保修申请吗?"

In [58]:
%%time
re3 = qa({"query": question3})

CPU times: user 3.73 s, sys: 1.15 s, total: 4.88 s
Wall time: 4.88 s


In [68]:
re3["result"]

' 如果客户已经设置了自动核准功能，平台可以自动核准保修申请。如果客户未设置自动核准，则需要手动核准。\n'

In [69]:
question4 = "自动核准后还需要进行单号提交吗?"

In [70]:
%%time
res4 = qa({"query": question4})

CPU times: user 1.54 s, sys: 575 ms, total: 2.12 s
Wall time: 2.12 s


In [71]:
res4["result"]

' 自动核准后，需要进行单号提交。\n'

In [72]:
question5 = "怎么查看保修进度?"

In [73]:
%%time
res5 = qa({"query": question5})

CPU times: user 2.44 s, sys: 826 ms, total: 3.27 s
Wall time: 3.27 s


In [74]:
res5["result"]

' 在厂家保修管理后台中，可以通过查看保修单号进入送修信息，查看保修及物流进度。\n'